<a href="https://colab.research.google.com/github/MaluGarcia18/predicciones-de-ventas-alimenticias/blob/main/predicciones_de_ventas_alimenticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd


In [23]:
filename = '/content/sales_predictions.csv'

sales_predictions = pd.read_csv(filename)
sales_predictions.head(3) 

# Primero: con HEAD.() examinar y  conocer el archivo.

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700


In [24]:
# Segundo: Revisar entradas duplicadas

#sales_predictions.duplicated()
sales_predictions.duplicated().sum()

0

No me arroja valores duplicados, por consiguiente, no se requiere de la eliminación de dichos datos:

Tercero: Remover columnas innecesarias

Para este punto se eliminará la columna que señala el año en que estableció la tienda, ya que no es un dato que influya directamente en la predición de las ventas.   

Lo mismo con el tipo de área donde se encuentra la tienda, el tamaño de la tienda 

Columnas a eliminar: Outlet_Establishment_Year, Outlet_Size y Outlet_Location_Type.

In [ ]:
# Tercero: Remover columnas innecesarias

sales_predictions = sales_predictions.drop(columns=['Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type'])
sales_predictions.head()


In [ ]:
# 4. Usar el método .nunique() 

sales_predictions.nunique()

Revisar que tipo de registros estan en las columnas de tipo categoricas para identificar faltas de ortografía, abreviaturas o errores tipográficos. Usando value_counts()

In [ ]:
# 5. Identificación de errores tipográficos en columnas categoricas.

sales_predictions['Outlet_Type'].value_counts()

In [28]:
sales_predictions['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [ ]:
sales_predictions['Item_Type'].value_counts()

In [30]:
# 6. Identificar valores NaN.

sales_predictions.isna().sum()

Item_Identifier         0
Item_Weight          1463
Item_Fat_Content        0
Item_Visibility         0
Item_Type               0
Item_MRP                0
Outlet_Identifier       0
Outlet_Type             0
Item_Outlet_Sales       0
dtype: int64

In [32]:
# Aca se busca hacer la relación de total de registros por columnas vs la cantidad de valores NaN.

sales_predictions['Item_Weight'].count()


7060

Imputar valores faltantes:

1. No hacer nada.
2. Imputación utilizando valores (media/mediana).
3. Imputación usando valores (más frecuentes) o cero/constante.
4. Imputación usando K-NN.
5. Imputación usando imputación multivariada por ecucación encadenada (MICE).
6. Imputación usando Deep Learning (Datawig)

In [35]:
# Considero que para la columna Item_Weight podré usar la imputación MICE.

#import sys
#from impyute.imputation.cs import mice
# NO ME DIO EL RESULTADO ESPERADO, PENDIENTE DE REVISAR

Obtener la media de las columnas con datos númericos:

In [36]:
# Las columnas con datos númericos son:  Item_Weight, Item_Visibility, Item_MRP, Item_Outlet_Sales
## Primero identifico las columnas. 
sales_predictions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Item_Identifier    8523 non-null   object 
 1   Item_Weight        7060 non-null   float64
 2   Item_Fat_Content   8523 non-null   object 
 3   Item_Visibility    8523 non-null   float64
 4   Item_Type          8523 non-null   object 
 5   Item_MRP           8523 non-null   float64
 6   Outlet_Identifier  8523 non-null   object 
 7   Outlet_Type        8523 non-null   object 
 8   Item_Outlet_Sales  8523 non-null   float64
dtypes: float64(4), object(5)
memory usage: 599.4+ KB


Item Weight: # Obtener mínimo, máximo y media para las columnas númericas.

In [39]:
sales_predictions['Item_Weight'].min()


4.555

In [45]:
sales_predictions['Item_Weight'].max()

21.35

In [44]:
sales_predictions['Item_Weight'].mean()

12.857645184135976

Item Visibility: # Obtener mínimo, máximo y media para las columnas númericas.

In [43]:
sales_predictions['Item_Visibility'].min()

0.328390948

In [46]:
sales_predictions['Item_Visibility'].max()

0.328390948

In [47]:
sales_predictions['Item_Visibility'].mean()

0.06613202877895108

Item MRP: # Obtener mínimo, máximo y media para las columnas númericas.

In [41]:
sales_predictions['Item_MRP'].min()

31.29

In [48]:
sales_predictions['Item_MRP'].max()

266.8884

In [49]:
sales_predictions['Item_MRP'].mean()

140.9927819781767

Item Outlet Sales: # Obtener mínimo, máximo y media para las columnas númericas.

In [42]:
sales_predictions['Item_Outlet_Sales'].min()

33.29

In [50]:
sales_predictions['Item_Outlet_Sales'].max()

13086.9648

In [51]:
sales_predictions['Item_Outlet_Sales'].mean()

2181.288913575032